# Create an EC2 Instance to install Data Fabric Server (single node)

## AWS Settings

This will create a security group in your default VPC, create a keypair and EC2 instance, and will enable access from this machine.

You need to provide AWS settings and credentials below.

EC2:
 - Ubuntu 20.04 (ami-0bd2099338bc55e6d for AWS eu-west-2 region, should be available in your configured region)
 - m5.4xlarge - 16 vCPU and 64GB memory
 - 150GB root (gp2) + 150GB data disks (gp3)

SG: Allow all traffic from my IP

In [ ]:
# Change these
ACCESS_KEY = ""
SECRET_KEY = ""
SESSION_TOKEN = ""

REGION = "eu-west-2"
AMI = "ami-0bd2099338bc55e6d" # Ubuntu 20.04 AMI (ami-0bd2099338bc55e6d for eu-west-2 / London)

# You can change these
TAG_PREFIX = "datafabric"
KEY_PAIR_NAME = TAG_PREFIX + "-ez-keypair"
SECURITY_GROUP_NAME = TAG_PREFIX + "-ez-securitygroup"
INSTANCE_NAME = TAG_PREFIX + "-ez-srv"
KEYFILE = TAG_PREFIX + "_key.pem"


In [ ]:
from botocore.config import Config

aws_config = Config(
    region_name = REGION,
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [ ]:
from distutils.command.config import config
from http import client
import boto3
from botocore.exceptions import ClientError
from paramiko import SSHClient, AutoAddPolicy, RSAKey

ec2 = boto3.client('ec2',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN,
    config=aws_config)
print("Done")

In [ ]:
# Helper functions

# Key Pair
import os


def create_key_pair():
    '''
    Get AWS Keypair with KEY_PAIR_NAME parameter
    
    params: None
    return: None
    '''
    try:
        keypairs = ec2.describe_key_pairs(KeyNames=[KEY_PAIR_NAME])
    except ClientError as e:
        if e.response['Error']['Code'] == "InvalidKeyPair.NotFound":
            print("Creating new keypair")
            key_pair = ec2.create_key_pair(KeyName=KEY_PAIR_NAME)
            with os.fdopen(os.open(KEYFILE, os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
                handle.write(key_pair['KeyMaterial'])
        else:
            print(e)
            raise

def find_instance():
    '''
    Find AWS Instances in Security Group SECURITY_GROUP_NAME parameter
    
    params: None
    return: InstanceId (str): AWS EC2 InstanceId if exists, else returns None
    '''
    instances = (instance
                 for reservation in ec2.describe_instances()['Reservations']
                 for instance in reservation['Instances'])
    for i in instances:
        if SECURITY_GROUP_NAME in [g['GroupName'] for g in i['SecurityGroups']]:
            return i['InstanceId']
    return None

def delete_instance(): # Will terminate all instances under the Security Group
    '''
    Deletes AWS Instances under SECURITY_GROUP_NAME parameter

    params: None
    return: None
    '''
    id = find_instance()
    while id:
        ec2.terminate_instances( InstanceIds = [id] )
        print("Shutting down existing instance %s ..." % id)
        ec2.get_waiter("instance_terminated").wait(InstanceIds=[id])
        print("Deleted Existing Instance %s" % id)
        id = find_instance()

# Security Group
def get_security_group():
    '''
    Returns SecurityGroup ID if found else returns None
    Searches by GroupName = SECURITY_GROUP_NAME

    params: none

    return: 
        SecurityGroupId (str): String representation of AWS Security Group
    '''
    SECURITY_GROUP_ID = None
    try:
        response = ec2.describe_security_groups(GroupNames=[SECURITY_GROUP_NAME])
        SECURITY_GROUP_ID = response.get("SecurityGroups", [{}])[0].get("GroupId", "")
        print("Found SG", SECURITY_GROUP_ID)
    except ClientError as e:
        print("Security Group %s not found" % SECURITY_GROUP_NAME)

    return SECURITY_GROUP_ID

def delete_security_group(sgid):
    '''
    Deletes Security Group and instances assigned to it!
    
    params: SecurityGroupId
    '''
    if sgid is not None:
        try:
            delete_instance()
            response = ec2.delete_security_group(GroupId=sgid)
            print("Deleted Existing Security Group %s" % sgid)
        except ClientError as e:
            print("ERROR", e)

def create_security_group():
    '''
    Creates AWS Security Group with SECURITY_GROUP_NAME parameter under default VPC
    Enables access from current host's public IP address for all traffic

    params: None
    return: None
    '''
    # Get default VPC
    response = ec2.describe_vpcs()
    vpc_id = response.get("Vpcs", [{}])[0].get("VpcId", "")

    sgid = get_security_group()
    if sgid is not None:
        delete_security_group(sgid)

    # Create SG with inbound rule - all traffic from my IP
    try:
        import requests
        MYIP = requests.get("https://ifconfig.me/ip").text + "/32"
        response = ec2.create_security_group(GroupName=SECURITY_GROUP_NAME,
                                                Description="Allow my IP",
                                                VpcId=vpc_id)
        SECURITY_GROUP_ID = response['GroupId']
        print("Created Security Group %s in %s." % (SECURITY_GROUP_ID, vpc_id))

        data = ec2.authorize_security_group_ingress(
            GroupId=SECURITY_GROUP_ID,
            IpPermissions=[
                {'IpProtocol': "-1",
                'IpRanges': [{'CidrIp': MYIP}]}
            ])
        print("Ingress Successfully Set for IP: %s" % data.get("SecurityGroupRules", [{}])[0].get('CidrIpv4', ''))
    except ClientError as e:
        print(e)

# EC2 Instance
def create_instance():
    '''
    Creates AWS EC2 instance with INSTANCE_NAME parameter

    params: None
    return: InstanceId (str): AWS EC2 InstanceId for newly created instance
    '''
    response = ec2.run_instances(
        ImageId=AMI,
        MinCount=1,
        MaxCount=1,
        InstanceType="m5.4xlarge",
        # InstanceType="t2.nano",
        KeyName=KEY_PAIR_NAME,
        SecurityGroupIds=[
            get_security_group(),
        ],
        BlockDeviceMappings=[
            {
                'DeviceName': "/dev/sda1",
                'Ebs': {
                    'DeleteOnTermination': True,
                    'VolumeSize': 120,
                    'VolumeType': "gp2"
                }
            },
            {
                'DeviceName': "/dev/xvda",
                'Ebs': {
                    'DeleteOnTermination': True,
                    'VolumeSize': 150,
                    'VolumeType': "gp2"
                }
            },
        ],
        TagSpecifications=[
            {
                'ResourceType': "instance",
                'Tags': [
                    {
                        'Key': "Name",
                        'Value': INSTANCE_NAME
                    },
                ]
            },
        ],
    )
    for i in response['Instances']:
        print("Waiting for instance %s to start..." % i['InstanceId'])
        ec2.get_waiter("instance_status_ok").wait(
            InstanceIds=[i['InstanceId']])
        return i['InstanceId']

def destroy_all():
    '''
    Destroy all resources created on AWS and deletes local files (ie, ssl key, ssl_truststore etc) 
        and removes /etc/hosts entries added during install

    params: None
    return: None
    '''
    instanceid = find_instance()
    while instanceid is not None:
        print("Delete for {}".format(instanceid))
        instance = ec2.describe_instances(InstanceIds=[]).get(
            'Reservations', [{}])[0].get('Instances', '')[0]
        pubip = instance['PublicIpAddress']
        pubdns = instance['PublicDnsName']
        prvip = instance['PrivateIpAddress']
        prvdns = instance['PrivateDnsName']
        print(pubip, pubdns, prvip, prvdns)
        !ssh-keygen -R $prvip 
        !ssh-keygen -R $prvdns
        !ssh-keygen -R $pubip 
        !ssh-keygen -R $pubdns
        sedptrn = "'/^%s/d'" % pubip
        os.system("sed {} /etc/hosts | sudo tee /etc/hosts".format(sedptrn))
        sedptrn = "'/%s/,+3d'" % pubip
        os.system("sed -i.bak {} ~/.ssh/config".format(sedptrn))
        delete_security_group(get_security_group())
        instanceid = find_instance()

    ec2.delete_key_pair(KeyName=KEY_PAIR_NAME)
    os.system("rm -f $KEYFILE ssl_truststore")


# SSH Client
client = SSHClient()
client.set_missing_host_key_policy(AutoAddPolicy())

def run_ssh_command(ip_address, username, keyfile, command):
    '''
    Runs given command on remote hosts

    params: 
        ip_address (str): IPv4 address or FQDN hostname to connect to
        username (str): username for remote host
        keyfile (str): path to SSH private key file
        command (str): any valid command to execute in remote system

    return: stdout (Generator): Returns each line returned from the command output
    '''
    try:
        client.connect(ip_address, port=22,
                       username=username, key_filename=keyfile)

        print("Running", command)
        stdin, stdout, stderr = client.exec_command(command)
        for line in iter(stdout.readline, ""):
            yield line
        for line in iter(stderr.readline, ""):
            yield line
            # print(line, end="")

        stdin.close()
        stdout.close()
        stderr.close()
        client.close()

    except BaseException as e:
        print(e)

def get_file(ip_address, username, keyfile, file):
    '''
    Copies remote file to local host with same name in default (running) directory

    params: 
        ip_address (str): IPv4 address or FQDN hostname to connect to
        username (str): username for remote host
        keyfile (str): path to SSH private key file
        file (str): full path to remote file

    return: None
    '''
    client.connect(ip_address, port=22,
                    username=username, key_filename=keyfile)
    sftp = client.open_sftp()

    print("getting", file, "to", os.path.basename(file))
    sftp.get(file, os.path.basename(file))

    sftp.close()
    client.close()

def put_file(ip_address, username, keyfile, file):
    '''
    Copies local file to remote host under /ubuntu/home directory

    params: 
        ip_address (str): IPv4 address or FQDN hostname to connect to
        username (str): username for remote host
        keyfile (str): path to SSH private key file
        file (str): full path to remote file

    return: None
    '''
    client.connect(ip_address, port=22,
                    username=username, key_filename=keyfile)
    sftp = client.open_sftp()

    print("put", file)
    sftp.put(localpath=file, remotepath='/home/ubuntu/{}'.format(os.path.basename(file)))


    sftp.close()
    client.close()

print("Done")

In [ ]:
# Setup AWS env - default VPC, SG & keypair

create_key_pair()
create_security_group()
print("Done")

In [ ]:
# Create VM
instanceid = create_instance()
instance = ec2.describe_instances(InstanceIds=[instanceid]).get(
    'Reservations', [{}])[0].get('Instances', '')[0]

print("Public IP: %s" % instance['PublicIpAddress'])
print("Public DNS: %s" % instance['PublicDnsName'])

In [ ]:
# Prepare the remote connection
instance = ec2.describe_instances(InstanceIds=[instanceid]).get(
    'Reservations', [{}])[0].get('Instances', '')[0]
prvdns = instance['PrivateDnsName']
prvip = instance['PrivateIpAddress']
pubip = instance['PublicIpAddress']
pubdns = instance['PublicDnsName']

# os.system("echo {} {} | sudo tee -a /etc/hosts".format(pubip, prvdns))

# ssh_config = """Host {}
#   User ubuntu
#   IdentityFile {}
# """.format(
#   pubdns,
#   os.getcwd() + "/" + KEYFILE
# )
# os.system("echo -e \"{}\" >> ~/.ssh/config".format(ssh_config))

print("Copying installation notebook to remote")
put_file(pubip, "ubuntu", KEYFILE, '01a-install-df-ubuntu.ipynb')
print("Copying private key")
put_file(pubip, "ubuntu", KEYFILE, KEYFILE)
# if os.path.isfile('LatestDemoLicense-M7.txt'):
#   print("Copying license file")
#   put_file(pubip, "ubuntu", KEYFILE, "LatestDemoLicense-M7.txt")

# for line in run_ssh_command(pubip, 'ubuntu', KEYFILE, 
#   "sudo apt update; sudo apt upgrade -y; sudo apt install -y python3-pip"):
#   print(line.strip())

print("Continue with remote connection to {} and follow ./01a-install-df-ubuntu.ipynb".format(pubdns))

In [ ]:
# After installation - copy secure files
for file in ["ssl_truststore", "ssl-client.xml", "maprtrustcreds.jceks", "maprtrustcreds.conf"]:
    get_file(pubip, "ubuntu", KEYFILE, file)


In [ ]:
# Uncomment and run following line to destroy the environment
# destroy_all()

#### Continue with [server install](./server-on-ubuntu.sh)

#### Continue to [create client on aws](./02a-create-dfclient-aws.ipynb)